# 모델 평가 (Model Evaluation)

이 노트북에서는 머신러닝 모델의 성능을 평가하는 다양한 지표와 방법을 학습합니다.

## 목차
1. 분류 평가 지표
   - 혼동 행렬 (Confusion Matrix)
   - 정확도, 정밀도, 재현율, F1-score
   - ROC 곡선과 AUC
   - Precision-Recall 곡선
2. 다중 분류 평가
3. 회귀 평가 지표
4. 학습 곡선

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer, load_iris, load_diabetes
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report,
    roc_curve, roc_auc_score, auc,
    precision_recall_curve, average_precision_score,
    mean_absolute_error, mean_squared_error, r2_score
)
from sklearn.preprocessing import label_binarize

# 시각화 설정
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.family'] = 'AppleGothic'  # MacOS용 한글 폰트
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

## 1. 분류 평가 지표

### 1.1 혼동 행렬 (Confusion Matrix)

In [ ]:
# 간단한 예시 데이터
y_true = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])
y_pred = np.array([1, 0, 1, 0, 0, 1, 1, 0, 1, 1])

# 혼동 행렬 계산
cm = confusion_matrix(y_true, y_pred)
print("혼동 행렬:")
print(cm)
print()

# 혼동 행렬 요소 추출
tn, fp, fn, tp = cm.ravel()
print(f"TN (True Negative): {tn}")
print(f"FP (False Positive): {fp} - Type I Error (위양성)")
print(f"FN (False Negative): {fn} - Type II Error (위음성)")
print(f"TP (True Positive): {tp}")

In [ ]:
# 혼동 행렬 시각화
fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Negative', 'Positive'])
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Confusion Matrix', fontsize=14, pad=20)
plt.show()

### 1.2 정확도, 정밀도, 재현율, F1-Score

In [ ]:
# 각 지표 계산
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("=== 분류 평가 지표 ===")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"  - (TP + TN) / (TP + TN + FP + FN)")
print(f"  - 전체 예측 중 정답 비율\n")

print(f"정밀도 (Precision): {precision:.4f}")
print(f"  - TP / (TP + FP)")
print(f"  - 양성으로 예측한 것 중 실제 양성의 비율\n")

print(f"재현율 (Recall/Sensitivity): {recall:.4f}")
print(f"  - TP / (TP + FN)")
print(f"  - 실제 양성 중 양성으로 예측한 비율\n")

print(f"F1-Score: {f1:.4f}")
print(f"  - 2 * (Precision * Recall) / (Precision + Recall)")
print(f"  - 정밀도와 재현율의 조화평균")

In [ ]:
# 수동 계산으로 검증
accuracy_manual = (tp + tn) / (tp + tn + fp + fn)
precision_manual = tp / (tp + fp) if (tp + fp) > 0 else 0
recall_manual = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_manual = 2 * precision_manual * recall_manual / (precision_manual + recall_manual) if (precision_manual + recall_manual) > 0 else 0

print("=== 수동 계산 검증 ===")
print(f"Accuracy:  {accuracy_manual:.4f}")
print(f"Precision: {precision_manual:.4f}")
print(f"Recall:    {recall_manual:.4f}")
print(f"F1-Score:  {f1_manual:.4f}")

### 1.3 실제 데이터셋으로 분류 평가 - Breast Cancer Dataset

In [ ]:
# 유방암 데이터셋 로드
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, test_size=0.2, random_state=42
)

# 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=10000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Breast Cancer Classification Results")
print("="*50)
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Features: {cancer.feature_names[:5]}... (total {len(cancer.feature_names)})")

In [ ]:
# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Malignant', 'Benign'])
disp.plot(ax=ax, cmap='RdYlGn', values_format='d')
plt.title('Confusion Matrix - Breast Cancer Classification', fontsize=14, pad=20)
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f"\nTrue Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

In [ ]:
# 분류 리포트
report = classification_report(y_test, y_pred, target_names=['Malignant', 'Benign'])
print("\n=== Classification Report ===")
print(report)

# 딕셔너리 형태로도 확인
report_dict = classification_report(y_test, y_pred, target_names=['Malignant', 'Benign'], output_dict=True)
print(f"\nBenign 클래스의 F1-score: {report_dict['Benign']['f1-score']:.4f}")
print(f"Malignant 클래스의 Recall: {report_dict['Malignant']['recall']:.4f}")

### 1.4 ROC 곡선과 AUC

In [ ]:
# 예측 확률
y_proba = model.predict_proba(X_test)[:, 1]

# ROC 곡선 계산
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

# ROC 곡선 시각화
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Random Classifier (AUC = 0.5)')
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
plt.ylabel('True Positive Rate (Sensitivity/Recall)', fontsize=12)
plt.title('ROC Curve - Breast Cancer Classification', fontsize=14, pad=20)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"AUC Score: {roc_auc:.4f}")
print(f"AUC Score (sklearn 직접 계산): {roc_auc_score(y_test, y_proba):.4f}")
print("\nAUC 해석:")
print("  - 1.0: 완벽한 분류기")
print("  - 0.5: 랜덤 분류기")
print("  - 0.0: 최악의 분류기")

### 1.5 Precision-Recall 곡선

In [ ]:
# PR 곡선 계산
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_proba)
ap = average_precision_score(y_test, y_proba)

# PR 곡선 시각화
plt.figure(figsize=(10, 6))
plt.plot(recall, precision, 'b-', linewidth=2, label=f'PR Curve (AP = {ap:.4f})')
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curve', fontsize=14, pad=20)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.show()

print(f"Average Precision (AP): {ap:.4f}")
print("\nROC vs PR 곡선:")
print("  - ROC: 불균형 데이터에서도 안정적, 전반적인 성능 평가")
print("  - PR: 불균형 데이터에서 더 민감, 양성 클래스 예측 성능에 집중")

In [ ]:
# ROC와 PR 곡선 동시 비교
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curve
axes[0].plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC (AUC = {roc_auc:.4f})')
axes[0].plot([0, 1], [0, 1], 'r--', linewidth=2)
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curve', fontsize=14)
axes[0].legend(loc='lower right', fontsize=11)
axes[0].grid(True, alpha=0.3)

# PR Curve
axes[1].plot(recall, precision, 'g-', linewidth=2, label=f'PR (AP = {ap:.4f})')
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curve', fontsize=14)
axes[1].legend(loc='best', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. 다중 분류 평가

In [ ]:
# Iris 데이터셋 로드 (3개 클래스)
iris = load_iris()
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# 모델 학습
model_iris = LogisticRegression(max_iter=1000, random_state=42)
model_iris.fit(X_train_iris, y_train_iris)
y_pred_iris = model_iris.predict(X_test_iris)

print("Iris Multi-class Classification")
print("="*50)
print(f"Classes: {iris.target_names}")
print(f"Features: {iris.feature_names}")

In [ ]:
# 다중 클래스 혼동 행렬
cm_iris = confusion_matrix(y_test_iris, y_pred_iris)
fig, ax = plt.subplots(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm_iris, display_labels=iris.target_names)
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Multi-class Confusion Matrix - Iris Dataset', fontsize=14, pad=20)
plt.show()

In [ ]:
# 다중 분류 지표
print("=== Multi-class Classification Metrics ===")
print(f"정확도: {accuracy_score(y_test_iris, y_pred_iris):.4f}\n")

# F1-score의 다양한 평균 방법
f1_macro = f1_score(y_test_iris, y_pred_iris, average='macro')
f1_weighted = f1_score(y_test_iris, y_pred_iris, average='weighted')
f1_micro = f1_score(y_test_iris, y_pred_iris, average='micro')

print(f"F1-Score (macro):    {f1_macro:.4f}  - 각 클래스의 F1을 단순 평균")
print(f"F1-Score (weighted): {f1_weighted:.4f}  - 각 클래스의 샘플 수로 가중 평균")
print(f"F1-Score (micro):    {f1_micro:.4f}  - 전체 TP, FP, FN을 합산하여 계산")

In [ ]:
# 분류 리포트
report_iris = classification_report(y_test_iris, y_pred_iris, target_names=iris.target_names)
print("\n=== Classification Report - Iris ===")
print(report_iris)

In [ ]:
# 다중 클래스 ROC 곡선
y_test_iris_bin = label_binarize(y_test_iris, classes=[0, 1, 2])
y_proba_iris = model_iris.predict_proba(X_test_iris)

plt.figure(figsize=(10, 6))
colors = ['blue', 'red', 'green']

for i, (color, name) in enumerate(zip(colors, iris.target_names)):
    fpr_i, tpr_i, _ = roc_curve(y_test_iris_bin[:, i], y_proba_iris[:, i])
    roc_auc_i = auc(fpr_i, tpr_i)
    plt.plot(fpr_i, tpr_i, color=color, linewidth=2,
             label=f'{name} (AUC = {roc_auc_i:.4f})')

plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Multi-class ROC Curves - Iris Dataset', fontsize=14, pad=20)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

## 3. 회귀 평가 지표

In [ ]:
# 간단한 예시
y_true_reg = np.array([3.0, -0.5, 2.0, 7.0, 4.5])
y_pred_reg = np.array([2.5, 0.0, 2.0, 8.0, 4.0])

# 회귀 지표 계산
mae = mean_absolute_error(y_true_reg, y_pred_reg)
mse = mean_squared_error(y_true_reg, y_pred_reg)
rmse = np.sqrt(mse)
r2 = r2_score(y_true_reg, y_pred_reg)

print("=== 회귀 평가 지표 ===")
print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"  - 평균적으로 예측이 실제값에서 {mae:.4f} 만큼 벗어남\n")

print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"  - 큰 오차에 더 큰 패널티\n")

print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"  - 타겟과 같은 단위로 해석 가능\n")

print(f"R² (Coefficient of Determination): {r2:.4f}")
print(f"  - 0~1, 1에 가까울수록 좋음")
print(f"  - 모델이 분산의 {r2*100:.1f}%를 설명")

In [ ]:
# 수동 계산으로 검증
print("\n=== 수동 계산 검증 ===")
mae_manual = np.mean(np.abs(y_true_reg - y_pred_reg))
mse_manual = np.mean((y_true_reg - y_pred_reg)**2)
rmse_manual = np.sqrt(mse_manual)
r2_manual = 1 - np.sum((y_true_reg - y_pred_reg)**2) / np.sum((y_true_reg - np.mean(y_true_reg))**2)

print(f"MAE:  {mae_manual:.4f}")
print(f"MSE:  {mse_manual:.4f}")
print(f"RMSE: {rmse_manual:.4f}")
print(f"R²:   {r2_manual:.4f}")

In [ ]:
# 실제 데이터셋으로 회귀 평가 - Diabetes Dataset
diabetes = load_diabetes()
X_train_diab, X_test_diab, y_train_diab, y_test_diab = train_test_split(
    diabetes.data, diabetes.target, test_size=0.2, random_state=42
)

# 선형 회귀 모델 학습
model_reg = LinearRegression()
model_reg.fit(X_train_diab, y_train_diab)
y_pred_diab = model_reg.predict(X_test_diab)

# 평가
mae_diab = mean_absolute_error(y_test_diab, y_pred_diab)
mse_diab = mean_squared_error(y_test_diab, y_pred_diab)
rmse_diab = np.sqrt(mse_diab)
r2_diab = r2_score(y_test_diab, y_pred_diab)

print("Diabetes Regression Results")
print("="*50)
print(f"MAE:  {mae_diab:.4f}")
print(f"MSE:  {mse_diab:.4f}")
print(f"RMSE: {rmse_diab:.4f}")
print(f"R²:   {r2_diab:.4f}")
print(f"\n해석: 모델이 타겟 분산의 {r2_diab*100:.1f}%를 설명합니다.")

In [ ]:
# 실제값 vs 예측값 시각화
plt.figure(figsize=(10, 6))
plt.scatter(y_test_diab, y_pred_diab, alpha=0.6, edgecolors='k', s=80)
plt.plot([y_test_diab.min(), y_test_diab.max()], 
         [y_test_diab.min(), y_test_diab.max()], 
         'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('실제값 (Actual)', fontsize=12)
plt.ylabel('예측값 (Predicted)', fontsize=12)
plt.title(f'실제값 vs 예측값 (R² = {r2_diab:.4f})', fontsize=14, pad=20)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 잔차 분석
residuals = y_test_diab - y_pred_diab

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 잔차 플롯
axes[0].scatter(y_pred_diab, residuals, alpha=0.6, edgecolors='k', s=80)
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('예측값 (Predicted)', fontsize=12)
axes[0].set_ylabel('잔차 (Residuals)', fontsize=12)
axes[0].set_title('Residual Plot', fontsize=14)
axes[0].grid(True, alpha=0.3)

# 잔차 분포
axes[1].hist(residuals, bins=20, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('잔차 (Residuals)', fontsize=12)
axes[1].set_ylabel('빈도 (Frequency)', fontsize=12)
axes[1].set_title('Residuals Distribution', fontsize=14)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"잔차 평균: {residuals.mean():.4f} (0에 가까워야 함)")
print(f"잔차 표준편차: {residuals.std():.4f}")

## 4. 학습 곡선 (Learning Curve)

In [ ]:
# 학습 곡선 계산
train_sizes, train_scores, val_scores = learning_curve(
    LogisticRegression(max_iter=10000, random_state=42),
    cancer.data, cancer.target,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# 평균 및 표준편차
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

# 학습 곡선 시각화
plt.figure(figsize=(10, 6))
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, 
                 alpha=0.2, color='blue')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, 
                 alpha=0.2, color='orange')
plt.plot(train_sizes, train_mean, 'o-', color='blue', linewidth=2, 
         label='Training Score')
plt.plot(train_sizes, val_mean, 'o-', color='orange', linewidth=2, 
         label='Validation Score')
plt.xlabel('Training Set Size', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Learning Curve - Breast Cancer Classification', fontsize=14, pad=20)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("학습 곡선 해석:")
print("  - 두 곡선이 모두 낮음 → 과소적합 (더 복잡한 모델 필요)")
print("  - 훈련 곡선 높고 검증 곡선 낮음 → 과적합 (정규화 필요)")
print("  - 두 곡선이 수렴 → 적절한 적합")

## 5. 평가 지표 선택 가이드

In [ ]:
# 종합 평가 함수
def evaluate_classification(y_true, y_pred, y_proba=None):
    """분류 모델 종합 평가"""
    print("=== 분류 평가 결과 ===")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"F1-Score:  {f1_score(y_true, y_pred, average='weighted'):.4f}")
    if y_proba is not None and len(np.unique(y_true)) == 2:
        print(f"ROC-AUC:   {roc_auc_score(y_true, y_proba):.4f}")

def evaluate_regression(y_true, y_pred):
    """회귀 모델 종합 평가"""
    print("=== 회귀 평가 결과 ===")
    print(f"MAE:  {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"MSE:  {mean_squared_error(y_true, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
    print(f"R²:   {r2_score(y_true, y_pred):.4f}")

# 테스트
print("Breast Cancer 모델 평가:")
evaluate_classification(y_test, y_pred, y_proba)

print("\nDiabetes 회귀 모델 평가:")
evaluate_regression(y_test_diab, y_pred_diab)

In [ ]:
# 평가 지표 요약 표
import pandas as pd

metrics_summary = pd.DataFrame({
    '지표': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'MAE', 'MSE', 'R²'],
    '분류/회귀': ['분류', '분류', '분류', '분류', '분류', '회귀', '회귀', '회귀'],
    '범위': ['0-1', '0-1', '0-1', '0-1', '0-1', '0-∞', '0-∞', '-∞-1'],
    '설명': [
        '전체 정답 비율',
        '양성 예측 중 실제 양성',
        '실제 양성 중 양성 예측',
        'Precision/Recall 조화평균',
        '분류기 전반적 성능',
        '평균 절대 오차',
        '평균 제곱 오차',
        '설명 분산 비율'
    ]
})

print("\n=== 평가 지표 요약 ===")
print(metrics_summary.to_string(index=False))

## 요약

### 분류 문제 지표 선택

1. **균형 데이터**: Accuracy, F1-score
2. **불균형 데이터**: Precision, Recall, F1-score, PR-AUC
   - 양성 클래스가 중요: Recall 중시 (암 진단, 사기 탐지)
   - 오탐이 비용: Precision 중시 (스팸 필터)
3. **확률 예측 품질**: ROC-AUC, PR-AUC
4. **다중 분류**: Macro/Weighted/Micro F1

### 회귀 문제 지표 선택

1. **기본**: MSE, RMSE, MAE
2. **이상치 민감도**: MAE (robust), MSE (sensitive)
3. **상대적 오차**: R²
4. **모델 비교**: R² (0~1 범위로 정규화)